In [2]:
import collections 
from itertools import chain
import torch 
import time 

from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter

## 项目folder
from dataset import SeqMultiLabelDataset
from process import Schema2Label
from models import BertHierClassifier
from evaluation import multilabel_inference, extract_multilabel, multilabel_evaluation

## 通用folder
from src.dataset.converter import data_loader 
from src.train_utils import set_seed, ModelSave, get_torch_device, EarlyStop, TrainParams
from src.metric import multilabel_metrics, multilabel_log
import transformers 
from transformers import BertTokenizer
transformers.logging.set_verbosity_error()
device = get_torch_device()

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [4]:
DIR = './trainsample'
schema = Schema2Label(DIR+'/duee_event_schema.json')

tp = TrainParams(
    log_steps = 50,
    save_steps = 10,
    epoch_size=20,
    max_seq_len=400,
    batch_size=20,
    loss_fn=nn.BCEWithLogitsLoss(),
    lr=5e-5,
    weight_decay=0.0,
    epsilon=1e-6,
    warmup_steps=100,
    dropout_rate=0.2,
    gradient_clip=5.0,
    early_stop_params = {
        'monitor':'f1',
        'mode':'max',
        'min_delta': 0,
        'patience':3,
        'verbose':False
    },
    pretrain_model = 'bert-base-chinese',
    continue_train=False,
    label2idx = schema.event_hier_label,
    idx2label = {j:i for i,j in schema.event_hier_label.items()},
    label_size=len(schema.event_hier_label),
    hier_relation = schema.event_hier_relid,
    hier_weight = 0.000001,
    num_layers=1
)

In [7]:
tokenizer = BertTokenizer.from_pretrained(tp.pretrain_model, do_lower_case=True)
train_dataset = SeqMultiLabelDataset(data_loader(DIR+'/train_event_hier.txt'),
                                tokenizer, tp.max_seq_len, tp.label2idx)
valid_dataset = SeqMultiLabelDataset(data_loader(DIR+'/valid_event_hier.txt'), 
                                tokenizer, tp.max_seq_len,  tp.label2idx)
train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)
train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=tp.batch_size)
valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=tp.batch_size)

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [8]:
tp.update({'num_train_steps': int(len(train_loader)*tp.epoch_size)})

CKPT = './checkpoint/event'
saver = ModelSave(CKPT, continue_train=False)
saver.init()
es = EarlyStop(**tp.early_stop_params)
global_step = 0
tb = SummaryWriter(CKPT)

model = BertHierClassifier(tp)
model.to(device)
optimizer, scheduler = model.get_optimizer()

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

In [9]:
for epoch_i in range(tp['epoch_size']):
    print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10}  | {'Elapsed':^9}")
    print("-"*60)

    # Measure the elapsed time of each epoch
    t0_epoch, t0_batch = time.time(), time.time()
    total_loss, batch_loss, batch_counts = 0, 0, 0

    model.train()
    for step, batch in enumerate(train_loader):
        global_step +=1
        batch_counts +=1

        #Forward propogate
        model.zero_grad()
        feature = {k:v.to(device) for k, v in batch.items()}
        logits = model(feature)
        loss = model.compute_loss(feature, logits)
        batch_loss += loss.item()
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), tp.gradient_clip)
        optimizer.step()
        scheduler.step()
        # Log steps for train loss logging
        if (step % tp.log_steps == 0 and step != 0) or (step == len(train_loader) - 1):
            time_elapsed = time.time() - t0_batch
            print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^9} | {time_elapsed:^9.2f}")
            tb.add_scalar('loss/batch_train', batch_loss / batch_counts, global_step=global_step)
            batch_loss, batch_counts = 0, 0
            t0_batch = time.time()

    # On Epoch End: calcualte train & valid loss and log overall metrics
    time_elapsed = time.time() - t0_epoch
    val_metrics = multilabel_metrics(model, valid_loader, device)
    avg_train_loss = total_loss / step

    print("-"*70)
    print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_metrics['val_loss']:^10.6f} | {time_elapsed:^9.2f}")
    multilabel_log(epoch_i, val_metrics)
    print("\n")
    if es.check(val_metrics):
        break 

 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   50    |   0.585779   |     -     |   41.57  
   1    |   100   |   0.266014   |     -     |   39.71  
   1    |   150   |   0.157661   |     -     |   39.68  
   1    |   200   |   0.128834   |     -     |   39.65  
   1    |   250   |   0.117124   |     -     |   39.66  
   1    |   300   |   0.108939   |     -     |   39.77  
   1    |   350   |   0.099315   |     -     |   39.74  
   1    |   400   |   0.090611   |     -     |   39.66  
   1    |   450   |   0.079278   |     -     |   39.72  
   1    |   500   |   0.073114   |     -     |   39.64  
   1    |   550   |   0.067858   |     -     |   39.69  
   1    |   595   |   0.063848   |     -     |   35.21  
----------------------------------------------------------------------
   1    |    -    |   0.154933   |  0.059847  |  473.71  


 Epoch  |  Val Acc  | Precision |  Recall   |  Val F1  
---------

In [10]:
test_dataset = SeqMultiLabelDataset(data_loader(DIR+'/test_event_hier.txt'), 
                                tokenizer, tp.max_seq_len,  tp.label2idx)
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset, sampler=test_sampler, batch_size=tp.batch_size)
test = pd.read_csv(DIR+'/test.csv')
pred = multilabel_inference(model, test_loader, device)
test['pred'] = pred 
test['pred_label'] = test['pred'].map(lambda x: extract_multilabel(x, tp.idx2label, 0.5))

In [11]:
valid = pd.read_csv(DIR+"/valid.csv")
pred = multilabel_inference(model, valid_loader, device)
valid['pred'] = pred
valid['pred_label'] = valid['pred'].map(lambda x: extract_multilabel(x, tp.idx2label, 0.5))

In [12]:
test['pred_label_leaf'] = test['pred_label'].map(lambda x: [i for i in x if i not in schema._event_hier_rel])
valid['pred_label_leaf'] = valid['pred_label'].map(lambda x: [i for i in x if i not in schema._event_hier_rel])
valid.to_csv('./trainsample/valid_event_hier_pred.csv')
test.to_csv('./trainsmaple/test_event_hier_pred.csv')